In [13]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

# Configuration

In [14]:
batch_size = 1
sequence_size = 64
num_heads = 4
head_size = 32
hidden_size = num_heads * head_size

# Initialize activations and weights using torch

In [15]:
torch_hidden_states = torch.randn((batch_size, sequence_size, hidden_size), dtype=torch.bfloat16)

torch_attention_mask = torch.zeros((1, 1, 1, sequence_size), dtype=torch.bfloat16)
torch_attention_mask[:, :, ::2, :] = -1e9

torch_query_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_query_bias = torch.randn((1, 1, 1, hidden_size), dtype=torch.bfloat16)
torch_key_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_key_bias = torch.randn((1, 1, 1, hidden_size), dtype=torch.bfloat16)
torch_value_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_value_bias = torch.randn((1, 1, 1, hidden_size), dtype=torch.bfloat16)
torch_output_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_output_bias = torch.randn((1, 1, 1, hidden_size), dtype=torch.bfloat16)

# Convert activations and weights to ttnn

In [16]:
hidden_states = ttnn.from_torch(torch_hidden_states)
attention_mask = ttnn.from_torch(torch_attention_mask)

query_weight = ttnn.from_torch(torch_query_weight)
query_bias = ttnn.from_torch(torch_query_bias)
key_weight = ttnn.from_torch(torch_key_weight)
key_bias = ttnn.from_torch(torch_key_bias)
value_weight = ttnn.from_torch(torch_value_weight)
value_bias = ttnn.from_torch(torch_value_bias)
output_weight = ttnn.from_torch(torch_output_weight)
output_bias = ttnn.from_torch(torch_output_bias)

hidden_states = ttnn.to_device(hidden_states, device)
attention_mask = ttnn.to_device(attention_mask, device)
query_weight = ttnn.to_device(query_weight, device)
query_bias = ttnn.to_device(query_bias, device, ttnn.l1_buffer_type)
key_weight = ttnn.to_device(key_weight, device)
key_bias = ttnn.to_device(key_bias, device, ttnn.l1_buffer_type)
value_weight = ttnn.to_device(value_weight, device)
value_bias = ttnn.to_device(value_bias, device, ttnn.l1_buffer_type)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, ttnn.l1_buffer_type)


# Write multi_head_attention using ttnn

In [17]:
def multi_head_attention(
    hidden_states,
    attention_mask,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    *,
    head_size,
):
    batch_size, sequence_size, hidden_size = hidden_states.shape
    num_heads = hidden_size // head_size

    query = hidden_states @ query_weight
    query = query + query_bias
    query = ttnn.reshape(query, (batch_size, sequence_size, num_heads, head_size))
    query = ttnn.permute(query, (0, 2, 1, 3))

    key = hidden_states @ key_weight
    key = key + key_bias
    key = ttnn.reshape(key, (batch_size, sequence_size, num_heads, head_size))
    key = ttnn.permute(key, (0, 2, 3, 1))

    value = hidden_states @ value_weight
    value = value + value_bias
    value = ttnn.reshape(value, (batch_size, sequence_size, num_heads, head_size))
    value = ttnn.permute(value, (0, 2, 1, 3))

    attention_scores = query @ key
    attention_scores = attention_scores * (1 / (head_size**0.5))
    if attention_mask is not None:
        attention_scores = attention_scores + attention_mask

    attention_probs = ttnn.softmax(attention_scores, dim=-1)

    context_layer = attention_probs @ value
    context_layer = ttnn.permute(context_layer, (0, 2, 1, 3))
    context_layer = ttnn.reshape(context_layer, (batch_size, sequence_size, hidden_size))

    self_output = context_layer @ output_weight
    self_output = self_output + output_bias

    return self_output

# Run using ttnn

In [18]:
output = multi_head_attention(
    hidden_states,
    attention_mask,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)

2023-10-26 22:08:28.638 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.649 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.664 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.674 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.693 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.704 | WARNING  | ttnn.core:reshape:476 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation
2023-10-26 22:08:28.751 | WARNING  | ttnn.core:reshape:476 - Given res

# Use output

In [19]:
print("Printing ttnn tensor")
output = ttnn.from_device(output)
print(output.shape)
print(output[0, 0, :1])

print("\n\n")
print("Printing torch tensor")
torch_output = ttnn.to_torch(output)
print(torch_output.shape)
print(torch_output[0, 0, :1])

Printing ttnn tensor
[1, 64, 128]
Tensor([ 28.75], dtype=bfloat16 )




Printing torch tensor
torch.Size([1, 64, 128])
tensor([28.7500], dtype=torch.bfloat16)


# Free tensor

In [20]:
ttnn.free(output)

# Free the device

In [21]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
